In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

In [4]:
%cd /content/drive/MyDrive/A_I/Ai

/content/drive/MyDrive/A_I/Ai


In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/bumper/bumper_data3.csv')
data_door_ed = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/door/door_ed_data3.csv')
data_door_scratch = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/door/door_scratch_data3.csv')
data_fender = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/fender/fender_ed_data3.csv')
data_frame_ed = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_ed_data3.csv')
data_frame_hd = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_hd_data3.csv')
data_frame_sealf = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_sealf_data3.csv')
data_frame_seamf = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_seamf_data3.csv')


# 0이 처음 출력되는 index 값 추출
first_zero_bumper = data_bumper[data_bumper['quality'] == 0].index[0]

first_zero_door_ed = data_door_ed[data_door_ed['quality'] == 0].index[0]
first_zero_door_scratch = data_door_scratch[data_door_scratch['quality'] == 0].index[0]

first_zero_fender = data_fender[data_fender['quality'] == 0].index[0]

first_zero_frame_ed = data_frame_ed[data_frame_ed['quality'] == 0].index[0]
first_zero_frame_hd = data_frame_hd[data_frame_hd['quality'] == 0].index[0]
first_zero_frame_sealf = data_frame_sealf[data_frame_sealf['quality'] == 0].index[0]
first_zero_frame_seamf = data_frame_seamf[data_frame_seamf['quality'] == 0].index[0]



# 일부 에러 데이터 정규화

In [6]:
import pandas as pd

def modify_csv(input_file, output_file, column_name, new_value, first_zero_frame_hd):
    # CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 조건 설정: 첫 번째 0이 나오기 전까지만
    row_condition = df.index < first_zero_frame_hd

    # 조건에 해당하는 값 수정
    df.loc[row_condition, column_name] = new_value

    # 수정된 데이터 저장
    df.to_csv(output_file, index=False)

#함수 사용 예시
modify_csv(
    "/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_hd_data3.csv",
    "./data/data/output/frame/frame_hd_data_modify.csv",
    "bboxes",
    "[]",
    first_zero_frame_hd
)

modify_csv(
    "/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_sealf_data3.csv",
    "./data/data/output/frame/frame_sealf_data_modify.csv",
    "bboxes",
    "[]",
    first_zero_frame_sealf
)

modify_csv(
    "/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_seamf_data3.csv",
    "./data/data/output/frame/frame_seamf_data_modify.csv",
    "bboxes",
    "[]",
    first_zero_frame_seamf
)


# 데이터 전처리
- 양품 : 75 * 4 = 300
- 불량품 : 300 * 4 = 1200 랜덤 샘플링

### index을 통해서 양품과 불량품의 위치를 구분

In [ ]:
# print(f"bumper 양품 : {len(data_bumper.iloc[:first_zero_bumper])}")
# print(f"bumper 불량품 : {len(data_bumper.iloc[first_zero_bumper:])}")

# print(f"door_ed 양품 : {len(data_door_ed.iloc[:first_zero_door_ed])}")
# print(f"door_ed 불양품 : {len(data_door_ed.iloc[first_zero_door_ed:])}")
# print(f"door_scratch 양품 : {len(data_door_scratch.iloc[:first_zero_door_scratch])}")
# print(f"door_scratch 불양품 : {len(data_door_scratch.iloc[first_zero_door_scratch:])}")

# print(f"data_fender 양품 : {len(data_fender.iloc[:first_zero_fender])}")
# print(f"data_fender 불량품 : {len(data_fender.iloc[first_zero_fender:])}")

# print(f"data_frame_ed 양품 : {len(data_frame_ed.iloc[:first_zero_frame_ed])}")
# print(f"data_frame_ed 불양품 : {len(data_frame_ed.iloc[first_zero_frame_ed:])}")
# print(f"data_frame_hd 양품 : {len(data_frame_hd.iloc[:first_zero_frame_hd])}")
# print(f"data_frame_hd 불양품 : {len(data_frame_hd.iloc[first_zero_frame_hd:])}")
# print(f"data_frame_sealf 양품 : {len(data_frame_sealf.iloc[:first_zero_frame_sealf])}")
# print(f"data_frame_sealf 불양품 : {len(data_frame_sealf.iloc[first_zero_frame_sealf:])}")
# print(f"data_frame_seamf 양품 : {len(data_frame_seamf.iloc[:first_zero_frame_seamf])}")
# print(f"data_frame_seamf 불양품 : {len(data_frame_seamf.iloc[first_zero_frame_seamf:])}")



### 양품 데이터 추출

In [7]:
import pandas as pd
data_door = pd.concat([data_door_ed.iloc[:first_zero_door_ed],
                      data_door_scratch.iloc[:first_zero_door_scratch]], ignore_index=True)
data_frame = pd.concat([data_frame_ed.iloc[:first_zero_frame_ed],
                        data_frame_hd.iloc[:first_zero_frame_hd], data_frame_sealf.iloc[:first_zero_frame_sealf], data_frame_seamf.iloc[:first_zero_frame_seamf]], ignore_index=True)

bumper_before_zero = data_bumper.iloc[:first_zero_bumper]
fender_before_zero = data_fender.iloc[:first_zero_fender]


# 75개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_ok_random_sample = bumper_before_zero.sample(
    n=min(75, len(bumper_before_zero)), replace=False, random_state=55)
fender_ok_random_sample = fender_before_zero.sample(
    n=min(75, len(fender_before_zero)), replace=False, random_state=55)
door_ok_random_sample = data_door.sample(n=75, replace=False)
frame_ok_random_sample = data_frame.sample(n=75, replace=False)
# 결과 출력
# print(bumper_ok_random_sample)
# print(len(bumper_ok_random_sample))

# print(fender_ok_random_sample)
# print(len(fender_ok_random_sample))

# print(door_ok_random_sample)
# print(len(door_ok_random_sample))

# print(frame_ok_random_sample)
# print(len(frame_ok_random_sample))

ok_data = pd.concat([bumper_ok_random_sample, fender_ok_random_sample,
                    door_ok_random_sample, frame_ok_random_sample], ignore_index=True, axis=0)

ok_data

,path,quality,part,bboxes
0,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
1,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
2,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
3,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
4,./data/data/source_data/bumper/scratch\205_101...,1,범퍼,[]
...,...,...,...,...
295,./data/data/source_data/frame/Exterior_damage\...,1,프레임,[]
296,./data/data/source_data/frame/Exterior_damage\...,1,프레임,[]
297,./data/data/source_data/frame/Exterior_damage\...,1,프레임,[]
298,./data/data/source_data/frame/Seam_failure\207...,1,프레임,"[[1550.843765162542, 860.1352657004832, 55.676..."


### 불량품 데이터 추출

In [8]:
import pandas as pd
data_fault_bumper = data_bumper.iloc[first_zero_bumper:]
data_fault_door = pd.concat([data_door_ed.iloc[first_zero_door_ed:],
                             data_door_scratch.iloc[first_zero_door_scratch:]], ignore_index=True)
data_fault_fender = data_fender.iloc[first_zero_fender:]
data_fault_frame = pd.concat([data_frame_ed.iloc[first_zero_frame_ed:],
                              data_frame_hd.iloc[first_zero_frame_hd:], data_frame_sealf.iloc[first_zero_frame_sealf:], data_frame_seamf.iloc[first_zero_frame_seamf:]], ignore_index=True)


# 300개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_fault_random_sample = data_fault_bumper.sample(n=300, replace=False)
fender_fault_random_sample = data_fault_fender.sample(n=300, replace=False)
door_fault_random_sample = data_fault_door.sample(n=300, replace=False)
frame_fault_random_sample = data_fault_frame.sample(n=300, replace=False)
# 결과 출력
# print(bumper_fault_random_sample)
# print(len(bumper_fault_random_sample))

# print(fender_fault_random_sample)
# print(len(fender_fault_random_sample))

# print(door_fault_random_sample)
# print(len(door_fault_random_sample))

# print(frame_fault_random_sample)
# print(len(frame_fault_random_sample))

fault_data = pd.concat([bumper_fault_random_sample, fender_fault_random_sample,
                        door_fault_random_sample, frame_fault_random_sample], ignore_index=True, axis=0)

fault_data

,path,quality,part,bboxes
0,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[1268.2883290267014, 558.3777100271003, 736.6..."
1,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[5.813953488372093, 1402.4548286604363, 1523...."
2,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[1576.496554694229, 1133.7209302325582, 2417...."
3,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[1302.325581395349, 1332.6230529595016, 220.9..."
4,./data/data/source_data/bumper/scratch\205_101...,0,범퍼,"[[447.86434108527123, 1347.9069767441858, 1185..."
...,...,...,...,...
1195,./data/data/source_data/frame/Hole_distortion\...,0,프레임,"[[1755.3606303584202, 647.2126315789474, 705.3..."
1196,./data/data/source_data/frame/Exterior_damage\...,0,프레임,"[[1793.3023255813953, 737.4478527607362, 410.2..."
1197,./data/data/source_data/frame/Exterior_damage\...,0,프레임,"[[1951.5348837209303, 886.1077844311377, 252.0..."
1198,./data/data/source_data/frame/Sealing_failure\...,0,프레임,"[[138.2450186288432, 191.92323584949833, 389.1..."


In [ ]:
# from sklearn.model_selection import train_test_split
# import pandas as pd



# # y에는 라벨, x에는 나머지 특성 정보
# y_ok = ok_data['quality']
# y_fault = fault_data['quality']

# # Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
# train_data_ok, test_data_ok = train_test_split(y_ok, test_size=0.2, stratify=y_ok, random_state=60)
# train_data_fault, test_data_fault = train_test_split(y_fault, test_size=0.2, stratify=y_fault, random_state=60)

# # 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
# train_data_80_ok, valid_data_ok = train_test_split(train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
# train_data_80_fault, valid_data_fault = train_test_split(train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# # CSV로 저장
# train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
# train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
# test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
# test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
# valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
# valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)
# # 결과 출력
# print(f"y_ok 총 데이터 개수: {len(y_ok)}")
# print(f"y_fault 총 데이터 개수: {len(y_fault)}")
# print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
# print(f"train_data_fault 학습 데이터 개수: {len(train_data_fault)}")
# print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
# print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
# print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
# print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")


In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd

# y에는 라벨, x에는 나머지 특성 정보
y_ok = ok_data['quality']
y_fault = fault_data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data_ok, test_data_ok = train_test_split(
    ok_data, test_size=0.2, stratify=y_ok, random_state=60
)
train_data_fault, test_data_fault = train_test_split(
    fault_data, test_size=0.2, stratify=y_fault, random_state=60
)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80_ok, valid_data_ok = train_test_split(
    train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
train_data_80_fault, valid_data_fault = train_test_split(
    train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# CSV로 저장
train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)

# 결과 출력
print(f"y_ok 총 데이터 개수: {len(y_ok)}")
print(f"y_fault 총 데이터 개수: {len(y_fault)}")
print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
print(f"train_data_80_fault 학습 데이터 개수: {len(train_data_80_fault)}")
print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")


y_ok 총 데이터 개수: 300
y_fault 총 데이터 개수: 1200
train_data_80_ok 학습 데이터 개수: 192
train_data_80_fault 학습 데이터 개수: 768
test_data_ok 테스트 데이터 개수: 60
test_data_fault 테스트 데이터 개수: 240
valid_data_ok 검증 데이터 개수: 48
valid_data_fault 검증 데이터 개수: 192


# 테스트 데이터

In [10]:
# 이미지 복사
import os
import shutil

def normalize_paths(data, path_column):
    """
    데이터프레임의 경로를 정규화하고 슬래시를 통일합니다.

    Args:
        data (pd.DataFrame): 경로가 포함된 데이터프레임
        path_column (str): 경로 열의 이름

    Returns:
        pd.DataFrame: 경로가 정규화된 데이터프레임
    """
    data[path_column] = data[path_column].apply(lambda x: os.path.normpath(x).replace("\\", "/"))
    return data

def copy_images(data, output_dir):
   # 이미지 디렉토리 생성
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    # 이미지 복사
    for index, row in data.iterrows():
        image_path = row['path']
        try:
            # 파일 존재 여부 확인
            if not os.path.exists(image_path):
                print(f"경고: 파일이 존재하지 않습니다 - {image_path}")
                continue

            # 복사 대상 파일 이름 생성
            destination = os.path.join(image_dir, os.path.basename(image_path))

            # 파일 복사
            shutil.copy(image_path, destination)
        except Exception as e:
            print(f"오류: {image_path} 복사 실패 - {e}")


# 데이터프레임의 경로 정규화 및 슬래시 통일

train_data_80_ok = normalize_paths(train_data_80_ok, 'path')
test_data_ok = normalize_paths(test_data_ok, 'path')
valid_data_o = normalize_paths(valid_data_ok,'path')
train_data_80_fault = normalize_paths(train_data_80_fault, 'path')
test_data_fault = normalize_paths(test_data_fault, 'path')
valid_data_fault = normalize_paths(valid_data_fault,'path')

# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_data_80_ok, './train')
copy_images(test_data_ok, './test')
copy_images(valid_data_ok,'./valid')
copy_images(train_data_80_fault, './train')
copy_images(test_data_fault, './test')
copy_images(valid_data_fault,'./valid')

### bboxes 데이터 추출

In [11]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_ok_bboxes = parse_bboxes(train_data_80_ok)
valid_ok_bboxes = parse_bboxes(valid_data_ok)
test_ok_bboxes = parse_bboxes(test_data_ok)
train_fault_bboxes = parse_bboxes(train_data_80_fault)
valid_fault_bboxes = parse_bboxes(valid_data_fault)
test_fault_bboxes = parse_bboxes(test_data_fault)

# 결과 출력 (예시)
print(train_ok_bboxes[:5])
print(valid_ok_bboxes[:5])
print(test_ok_bboxes[:5])
print(train_fault_bboxes[:5])
print(valid_fault_bboxes[:5])
print(test_fault_bboxes[:5])

[[], [], [], [[1117.2558139534883, 494.2222222222222, 60.488372093023315, 74.6666666666667]], []]
[[], [], [], [[1364.7998091668005, 1594.463161991298, 31.551786872151833, 152.16108489485487]], []]
[[[1429.9108527131782, 1109.3333333333333, 52.907649655469534, 361.9117512574301]], [], [], [], []]
[[[1517.4418604651164, 880.1742919389978, 2609.302325581395, 1298.474945533769]], [[1537.889230309307, 547.4474853801169, 894.2037929465072, 502.9716762965297]], [[1840.1860465116279, 655.5092024539878, 457.11627906976736, 532.6012269938647]], [[1518.6815125849362, 720.0133384292913, 514.0294525791949, 198.7854233170517]], [[475.7984831222294, 819.8504781251568, 217.15391881413163, 322.7164444266451]]]
[[[1596.2790697674418, 1221.7674418604652, 1251.441829237827, 497.3023255813954]], [[1724.5032019730731, 773.2645161290322, 648.9851701199501, 509.6516129032258]], [[1980.8372093023256, 1160.372093023256, 457.1162790697674, 205.11627906976742], [638.7906976744187, 662.232558139535, 673.953488372

# .yaml label 데이터 생성

### 1. 디렉토리 경로 전처리

In [12]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_ok_image_paths = train_data_80_ok.iloc[:, 0].tolist()
valid_ok_image_paths = valid_data_ok.iloc[:, 0].tolist()
test_ok_image_paths = test_data_ok.iloc[:, 0].tolist()
train_fault_image_paths = train_data_80_fault.iloc[:, 0].tolist()
valid_fault_image_paths = valid_data_fault.iloc[:, 0].tolist()
test_fault_image_paths = test_data_fault.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes_ok, train_dirs_ok, train_names_ok = extract_image_info(train_ok_image_paths)
valid_sizes_ok, valid_dirs_ok, valid_names_ok = extract_image_info(valid_ok_image_paths)
test_sizes_ok, test_dirs_ok, test_names_ok = extract_image_info(test_ok_image_paths)
train_sizes_fault, train_dirs_fault, train_names_fault = extract_image_info(train_fault_image_paths)
valid_sizes_fault, valid_dirs_fault, valid_names_fault = extract_image_info(valid_fault_image_paths)
test_sizes_fault, test_dirs_fault, test_names_fault = extract_image_info(test_fault_image_paths)

# 결과 출력
# ok
print("train_sizes_ok 이미지 크기:", train_sizes_ok[:5])
print("train_dirs_ok 이미지 디렉토리:", train_dirs_ok[:5])
print("train_names_ok 파일 이름:", train_names_ok[:5])
print()
print("valid_sizes_ok 이미지 크기:", valid_sizes_ok[:5])
print("valid_dirs_ok 이미지 디렉토리:", valid_dirs_ok[:5])
print("valid_names_ok 파일 이름:", valid_names_ok[:5])
print()
print("test_sizes_ok 이미지 크기:", test_sizes_ok[:5])
print("test_dirs_ok 이미지 디렉토리:", test_dirs_ok[:5])
print("test_names_ok 파일 이름:", test_names_ok[:5])
# fault
print("train_sizes_fault 이미지 크기:", train_sizes_fault[:5])
print("train_dirs_fault 이미지 디렉토리:", train_dirs_fault[:5])
print("train_names_fault 파일 이름:", train_names_fault[:5])
print()
print("valid_sizes_fault 이미지 크기:", valid_sizes_fault[:5])
print("valid_dirs_fault 이미지 디렉토리:", valid_dirs_fault[:5])
print("valid_names_fault 파일 이름:", valid_names_fault[:5])
print()
print("test_sizes_fault 이미지 크기:", test_sizes_fault[:5])
print("test_dirs_fault 이미지 디렉토리:", test_dirs_fault[:5])
print("test_names_fault 파일 이름:", test_names_fault[:5])

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


train_sizes_ok 이미지 크기: [(4032, 3024), (4032, 1908), (4032, 1908), (2448, 2048), (4032, 1816)]
train_dirs_ok 이미지 디렉토리: ['data/data/source_data/bumper/scratch/', 'data/data/source_data/bumper/scratch/', 'data/data/source_data/fender/Exterior_damage/', 'data/data/source_data/frame/Hole_distortion/', 'data/data/source_data/door/scratch/']
train_names_ok 파일 이름: ['205_101_10_1419f342-b811-454f-a371-57c02c6343c8', '205_101_10_60da5008-10b2-40b9-a0f6-e7dc22d2f11c', '209_102_10_319c5ffd-6e52-4711-a74f-bee9f0431a4f', '207_213_10_3a770e39-5c72-45f4-b70f-9f0f89b889b4', '204_101_10_617eb0f7-223d-4aa6-b3a8-0c46977518c7']

valid_sizes_ok 이미지 크기: [(4032, 1960), (4032, 1908), (4000, 1800), (2448, 2048), (4032, 1908)]
valid_dirs_ok 이미지 디렉토리: ['data/data/source_data/fender/Exterior_damage/', 'data/data/source_data/bumper/scratch/', 'data/data/source_data/door/scratch/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/fender/Exterior_damage/']
valid_names_ok 파일 이름: ['209_102_10_daa6a82

### 3. 부품 상태 전처리

In [13]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 2
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 3
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 4
    return None  # 기본적으로 반환할 값이 없으면 None

# 예시 부품과 상태
parts_and_status_train_ok = []
parts_and_status_valid_ok = []
parts_and_status_test_ok = []
for quality, part in zip(train_data_80_ok.iloc[:, 1], train_data_80_ok.iloc[:, 2]):
    parts_and_status_train_ok.append({"part": part, "status": quality})

for quality, part in zip(test_data_ok.iloc[:, 1], test_data_ok.iloc[:, 2]):
    parts_and_status_test_ok.append({"part": part, "status": quality})

for quality, part in zip(valid_data_ok.iloc[:, 1], valid_data_ok.iloc[:, 2]):
    parts_and_status_valid_ok.append({"part": part, "status": quality})

parts_and_status_train_fault = []
parts_and_status_valid_fault = []
parts_and_status_test_fault = []
for quality, part in zip(train_data_80_fault.iloc[:, 1], train_data_80_fault.iloc[:, 2]):
    parts_and_status_train_fault.append({"part": part, "status": quality})

for quality, part in zip(test_data_fault.iloc[:, 1], test_data_fault.iloc[:, 2]):
    parts_and_status_test_fault.append({"part": part, "status": quality})

for quality, part in zip(valid_data_fault.iloc[:, 1], valid_data_fault.iloc[:, 2]):
    parts_and_status_valid_fault.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train_ok}")
print(f"parts_and_status_test : {parts_and_status_test_ok}")
print(f"parts_and_status_valid : {parts_and_status_valid_ok}")

print(f"parts_and_status_train : {parts_and_status_train_fault}")
print(f"parts_and_status_test : {parts_and_status_test_fault}")
print(f"parts_and_status_valid : {parts_and_status_valid_fault}")

# 라벨링 결과를 저장할 리스트
labels_train_ok = []
labels_valid_ok = []
label_test_ok = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_ok:
    label = get_label(item["part"], item["status"])
    labels_train_ok.append(label)
for item in parts_and_status_test_ok:
    label = get_label(item["part"], item["status"])
    labels_valid_ok.append(label)
for item in parts_and_status_valid_ok:
    label = get_label(item["part"], item["status"])
    label_test_ok.append(label)

# # 결과 출력
print("labels_train_ok 라벨링 결과 :", labels_train_ok)
print("labels_valid_ok 라벨링 결과 :", labels_valid_ok)
print("label_test_ok 라벨링 결과 :", label_test_ok)

# 라벨링 결과를 저장할 리스트
labels_train_fault = []
labels_valid_fault = []
label_test_fualt = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_fault:
    label = get_label(item["part"], item["status"])
    labels_train_fault.append(label)
for item in parts_and_status_test_fault:
    label = get_label(item["part"], item["status"])
    labels_valid_fault.append(label)
for item in parts_and_status_valid_fault:
    label = get_label(item["part"], item["status"])
    label_test_fualt.append(label)

# # 결과 출력
print("labels_train_fault 라벨링 결과 :", labels_train_fault)
print("labels_valid_fault 라벨링 결과 :", labels_valid_fault)
print("label_test_fualt 라벨링 결과 :", label_test_fualt)

parts_and_status_train : [{'part': '범퍼', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': 

In [14]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.0 0.0 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_ok_bboxes, labels_train_ok, train_sizes_ok, train_names_ok)
save_yolo_labels(train_output_dir, train_fault_bboxes, labels_train_fault, train_sizes_fault, train_names_fault)
# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_ok_bboxes, labels_valid_ok, valid_sizes_ok, valid_names_ok)
save_yolo_labels(valid_output_dir, valid_fault_bboxes, labels_valid_fault, valid_sizes_fault, valid_names_fault)

# 라벨을 저장할 디렉토리
test_output_dir = './test/labels'
os.makedirs(test_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(test_output_dir, test_ok_bboxes, label_test_ok, test_sizes_ok, test_names_ok)
save_yolo_labels(test_output_dir, test_fault_bboxes, label_test_fualt, test_sizes_fault, test_names_fault)

In [18]:
# YAML 파일에 저장
yaml_data = {
    "train": "/content/drive/MyDrive/A_I/Ai/train/images",
    "test": "/content/drive/MyDrive/A_I/Ai/test/images",
    "val": "/content/drive/MyDrive/A_I/Ai/valid/images",
    "nc": 5,
    "names": ["ok",
              "bumper_faulty",
              "door_faulty",
              "fender_faulty",
              "frame_faulty"]
}

with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

In [16]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 42.0 MB/s eta 0:00:00


## 모델 코드 작성

In [19]:
import os
from ultralytics import YOLO

# 데이터 설정 파일 경로
data_yaml = "data.yaml"

# YOLO 모델 정의 (랜덤 초기화로 학습 시작)
model = YOLO("yolov8m.pt")  # 'yolov5n'은 작은 모델, 필요에 따라 yolov5s, yolov5m 선택 가능
# model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# 학습 설정
model.train(
    data=data_yaml,           # 데이터 설정 파일 경로
    epochs=20,               # 학습 에폭 수
    batch=20,            # 배치 크기
    imgsz=640,                # 입력 이미지 크기
    device=0,                 # GPU 사용
    workers=8,                # 데이터 로드 병렬 처리 워커 수
    )

model.save("model_8m_b20_w8.pt")

Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=20, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=

100%|██████████| 755k/755k [00:00<00:00, 38.8MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

train: Scanning /content/drive/MyDrive/A_I/Ai/train/labels... 199 images, 0 backgrounds, 0 corrupt:  12%|█▏        | 199/1640 [02:06<09:00,  2.67it/s]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
train: Scanning /content/drive/MyDrive/A_I/Ai/train/labels... 1639 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1640/1640 [11:18<00:00,  2.42it/s]

train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_037c081e-5304-4903-8ef3-82f691f87d30.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_0af1a562-dca3-4fc2-b8d6-99b407bca992.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_0ca8b8be-b301-4659-9779-860699a71b92.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_19c3cea1-a742-4da0-9ee0-5f8687c62bce.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_1bddf87a-2fec-45ab-9021-c9c6051939e0.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_1bf32c03-120f-4e95-bbfe-031d47c49d5c.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_1fdf0cec-b84d-43f9-8a52-02d98ae6e870.jpg: corrupt JP

...

       2/20      8.37G      2.136      2.974       2.04         40        640: 100%|██████████| 82/82 [04:06<00:00,  3.00s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:54<00:00,  4.54s/it]


                   all        441        563     0.0278     0.0415    0.00592     0.0024

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/20      8.38G      2.239       3.01      2.132         65        640:  45%|████▌     | 37/82 [01:40<02:01,  2.70s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-ba808162a0ce>", line 12, in <cell line: 0>
    model.train(
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/model.py", line 806, in train
    self.trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 207, in train
    self._do_train(world_size)
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/engine/trainer.py", line 363, in _do_train
    for i, batch in pbar:
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/data/build.py", line 48, in __iter__
    yield next(self.iterator)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/datalo

TypeError: object of type 'NoneType' has no len()

In [ ]:
# from ultralytics import YOLO

# # YOLO 모델 객체 생성
# model = YOLO("yolov8s.pt")  # 사전 학습된 YOLOv8 모델 로드

# # 모델 학습 시작
# results = model.train(
#     data="data.yaml",       # YAML 파일 경로
#     epochs=20,              # 학습 에포크 수
#     imgsz=640,              # 입력 이미지 크기 (기본값: 640)
#     batch=16,               # 배치 크기
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO", # 학습 결과를 저장할 프로젝트 이름
#     name="part_damage_detection",   # 저장될 실행 이름
#     workers=4,              # 데이터 로드 병렬 처리 수
#     device=0                # GPU ID (0은 첫 번째 GPU를 의미, -1은 CPU)
# )


In [ ]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (랜덤 초기화로 학습 시작)
# model = YOLO("yolov8s.pt")
# # model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# # 학습 설정
# model.train(
#     data=data_yaml,          # 데이터 설정 파일 경로
#     epochs=50,               # 학습 에폭 수
#     batch=16,                # 배치 크기
#     imgsz=640,               # 입력 이미지 크기
#     device=0,                # GPU 사용
#     workers=4,               # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",     # 결과 저장 디렉토리
#     name="part_damage_detection",       # 실험 이름
#     conf=0.5,
#     lr0=0.001,  # 초기 학습률
#     optimizer='AdamW',  # 옵티마이저 설정 (SGD, Adam, AdamW 등)
#     weight_decay=0.01   # 가중치 감소 설정
# )
# model.save('part_damage_detection.pt')

In [ ]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8n.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=70,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기
#     device=0,
#     workers=3,
#     project="YOLO1",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0001,
#     optimizer='AdamW',
#     weight_decay=0.01,
#     patience=10,
# )

# model.save('part_damage_detection2.pt')

In [ ]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=50,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기 줄이기
#     device=0,
#     workers=4,
#     project="YOLO2",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0005,
#     optimizer='AdamW',
#     weight_decay=0.005,
#     patience=10,
# )

# model.save('part_damage_detection3.pt')

In [ ]:
# # 모델 평가
# metrics = model.val()

### 모델 로드 및 추론

In [ ]:
# 모델 로드 및 추론
model = YOLO("C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_50_16_944.pt")
results = model.predict(source="C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\test\\images", save=True, imgsz=640)



In [ ]:
count = 0
for result in results:
    # 바운딩 박스가 없는 경우 'no detections'으로 간주
    if result.boxes is None or len(result.boxes.xywh) == 0:
        count += 1

print(f"no detections : {count}")
print(f"len(results) : {len(results)}")


NameError: name 'results' is not defined

In [ ]:
# import cv2
# from ultralytics import YOLO

# # YOLOv8 모델 로드
# model_path = "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_50_16_944.pt"  # 학습한 모델 경로
# model = YOLO(model_path)
# model.to("cuda")

# # 카메라 열기
# cap = cv2.VideoCapture(0)  # 기본 카메라 (ID 0)
# if not cap.isOpened():
#     print("카메라를 열 수 없습니다.")
#     exit()

# # 카메라 프레임 크기 설정 (선택 사항)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# # 색상 설정 (바운딩 박스와 텍스트)
# box_color = (0, 255, 0)  # 녹색
# text_color = (255, 255, 255)  # 흰색

# print("Press 'q' to exit...")

# while True:
#     # 프레임 읽기
#     ret, frame = cap.read()
#     if not ret:
#         print("프레임을 읽을 수 없습니다.")
#         break
#     flip_frame = cv2.flip(frame, 1)
#     # 모델 추론 (YOLOv8)
#     results = model(source=frame, conf=0.5, show=False,
#                     verbose=False)  # 이미지에 대해 추론
#     detections = results[0]  # 첫 번째 결과 가져오기

#     # 바운딩 박스와 라벨 그리기
#     for box in detections.boxes:
#         # 바운딩 박스 좌표 (정규화된 값 아님)
#         x1, y1, x2, y2 = map(int, box.xyxy[0])  # 좌상단(x1, y1), 우하단(x2, y2)
#         conf = box.conf[0]  # 신뢰도
#         cls = int(box.cls[0])  # 클래스 ID
#         label = f"{model.names[cls]} {conf:.2f}"  # 클래스 이름과 신뢰도

#         # 바운딩 박스와 라벨 추가
#         cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)  # 바운딩 박스
#         cv2.putText(frame, label, (x1, y1 - 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

#     # 프레임 화면에 표시
#     cv2.imshow("YOLOv8 Real-Time Detection", flip_frame)

#     # 'q' 키를 누르면 종료
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # 자원 해제
# cap.release()
# cv2.destroyAllWindows()

## 손실 그래프

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

## 데이터 검증

In [ ]:
# ## data evaluation

# from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
# pred=model(y_test)


# # 혼동 행렬
# confmat = confusion_matrix(y_true=y_test, y_pred=pred)
# print('혼동 행렬: ', confmat)


# # 정확도
# accuracy=accuracy_score(y_true=y_test, y_pred=pred)
# print('정확도: ', accuracy)


# # 정밀도(precision)
# precision = precision_score(y_true=y_test, y_pred=pred)
# print('정밀도: ', precision)


# # 재현율(recall)
# recall = recall_score(y_true=y_test, y_pred=pred)
# print('재현율: ', recall)


# # roc auc
# fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

# plt.plot(fpr, tpr)
# plt.xlabel('FP Rate')
# plt.ylabel('TP Rate')

# plt.show()

# print('auc 점수: ', roc_auc_score(y_test, pred))

In [ ]:
%pip install numpy

In [ ]:
%pip install scikit-learn

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred = []

for result in results:
    if len(result.boxes) > 0:  # Detection이 있을 경우
        # 각 box에서 confidence가 가장 높은 클래스 선택
        pred.append(int(result.boxes[0].cls))
    else:
        pred.append(0)  # No Detection은 '0'으로 처리


# y_test 생성
y_test = []

for i in range(len(test_test_20)):
    if test_test_20.iloc[i, 1] == 1:  # 열 1에 접근
        y_test.append(0)
    elif test_test_20.iloc[i, 2] == '범퍼':  # 열 2에 접근
        y_test.append(1)
    elif test_test_20.iloc[i, 2] == '도어':
        y_test.append(2)
    elif test_test_20.iloc[i, 2] == '휀더':
        y_test.append(3)
    elif test_test_20.iloc[i, 2] == '프레임':
        y_test.append(4)


## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt


# 위의 ROC 커브 코드 재사용 가능

print(y_test)
print(pred)

# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat, sep='\n')


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도:', accuracy)

# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred, average='macro')
print('정밀도: ', precision, sep='/n')


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred, average='macro')
print('재현율: ', recall)